In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np

In [ ]:
import numpy as np
import cv2
import tensorflow as tf

minValue = 70

def func(path):    
    frame = cv2.imread(path)
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),2)

    th3 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
    ret, res = cv2.threshold(th3, minValue, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    return res



# Loading dataset 

# Dense model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the image dimensions and batch size
image_width, image_height = 200, 200
batch_size = 32

# Specify the paths to the directories
train_dir = '/content/drive/Shareddrives/BA865 - Team Project/Image_dataset/train'
validation_dir = '/content/drive/Shareddrives/BA865 - Team Project/Image_dataset/validation'

# Create an ImageDataGenerator for data augmentation and rescaling
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Load the training and validation data from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='categorical'
)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 84 images belonging to 34 classes.
Found 34 images belonging to 34 classes.


In [ ]:
# Define the simple dense neural network model
model = Sequential([
    Flatten(input_shape=(image_width, image_height, 3)),
    Dense(128, activation='relu'),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Evaluate the model
scores = model.evaluate(validation_generator)
print(f"Validation accuracy: {scores[1]*100:.2f}%")


Epoch 1/10
3/3 [==============================] - 6s 2s/step - loss: 60.2868 - accuracy: 0.0476 - val_loss: 78.8252 - val_accuracy: 0.0294
Epoch 2/10
3/3 [==============================] - 2s 788ms/step - loss: 64.5286 - accuracy: 0.0476 - val_loss: 56.9160 - val_accuracy: 0.0000e+00
Epoch 3/10
3/3 [==============================] - 2s 756ms/step - loss: 42.9667 - accuracy: 0.0714 - val_loss: 41.9193 - val_accuracy: 0.0588
Epoch 4/10
3/3 [==============================] - 3s 1s/step - loss: 31.9483 - accuracy: 0.1548 - val_loss: 31.1110 - val_accuracy: 0.0588
Epoch 5/10
3/3 [==============================] - 3s 834ms/step - loss: 22.3245 - accuracy: 0.1548 - val_loss: 24.9248 - val_accuracy: 0.0294
Epoch 6/10
3/3 [==============================] - 2s 871ms/step - loss: 16.8604 - accuracy: 0.1310 - val_loss: 15.8458 - val_accuracy: 0.0294
Epoch 7/10
3/3 [==============================] - 2s 895ms/step - loss: 9.5141 - accuracy: 0.2143 - val_loss: 10.0977 - val_accuracy: 0.0588
Epoch 8/1

## Prediction on testset 

In [ ]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def predict_single_image(model, image_path, target_size, class_labels):
    image = load_img(image_path, target_size=target_size)
    image_array = img_to_array(image)
    image_array = np.expand_dims(image_array, axis=0)
    prediction = model.predict(image_array)
    predicted_class_index = np.argmax(prediction, axis=-1)[0]
    predicted_class_label = class_labels[predicted_class_index]
    return predicted_class_label

def predict_multiple_images(model, folder_path, class_labels, target_size=(200, 200)):
    predicted_labels = {}
    for root, dirs, files in os.walk(folder_path):
        for image_filename in files:
            # Construct the full path to the image file
            image_path = os.path.join(root, image_filename)
            if os.path.isfile(image_path):  # Ensure that the item is a file
                predicted_class_label = predict_single_image(model, image_path, target_size, class_labels)
                predicted_labels[image_filename] = predicted_class_label
    return predicted_labels

# Define the path to the folder containing the test images
test_dir = '/content/drive/Shareddrives/BA865 - Team Project/Image_dataset/test'

# Get the class labels from the training generator
class_labels = list(train_generator.class_indices.keys())

# Use the trained model to predict the classes of the test images
predicted_labels = predict_multiple_images(model, test_dir, class_labels)
print('Predicted class labels for test images:')
for image_filename, predicted_class_label in predicted_labels.items():
    print(f'{image_filename}: {predicted_class_label}')


1/1 [==============================] - 0s 35ms/step
Predicted class labels for test images:
yes.0.png: phone
eat.0.png: phone


## Pre-train cnn

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten

# Load the VGG16 pretrained model without the top dense layers
conv_base = VGG16(
    weights="imagenet",
    include_top=False,  # Exclude the top dense layers
    input_shape=(200, 200, 3)  # The input shape for the base model (grayscale images will need to be converted to RGB)
)

# Get the output of the base model
x = conv_base.output

# Add a global average pooling layer
x = GlobalAveragePooling2D()(x)

# Add a dropout layer (optional)
x = Dropout(0.25)(x)

# Add a dense layer with 128 neurons and ReLU activation
x = Dense(128, activation='relu')(x)

# Add a dropout layer with 50% dropout rate
x = Dropout(0.5)(x)

# Output layer with the number of neurons equal to the number of classes, using softmax activation
num_classes = train_generator.num_classes  # Number of classes to predict
predictions = Dense(num_classes, activation='softmax')(x)

# Create a new model combining the base model and custom layers
model = Model(inputs=conv_base.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()

# Train the model using the data generators
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Evaluate the model on the validation data
scores = model.evaluate(validation_generator)
print(f"Validation accuracy: {scores[1]*100:.2f}%")


58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 200, 200, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 200, 200, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 100, 100, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 100, 100, 128)     147584    
                                                              

KeyboardInterrupt: ignored

# Conv2 model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Define the CNN model
model = Sequential([
    # Convolutional layer with 32 filters, each with size 3x3, and ReLU activation
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(200, 200, 3)),
    # Max pooling layer with pool size 2x2
    MaxPooling2D(pool_size=(2, 2)),
    # Dropout layer with 25% dropout rate
    Dropout(0.25),
    # Another convolutional layer with 64 filters, each with size 3x3, and ReLU activation
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    # Max pooling layer with pool size 2x2
    MaxPooling2D(pool_size=(2, 2)),
    # Dropout layer with 25% dropout rate
    Dropout(0.25),
    # Flatten layer to convert 3D feature maps to 1D feature vectors
    Flatten(),
    # Dense layer with 128 neurons and ReLU activation
    Dense(128, activation='relu'),
    # Dropout layer with 50% dropout rate
    Dropout(0.5),
    # Output layer with the number of neurons equal to the number of classes, using softmax activation
    Dense(len(train_generator.class_indices), activation='softmax')  # num_classes should be replaced with the actual number of classes
])


In [ ]:

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 99, 99, 32)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 99, 99, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 97, 97, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 48, 48, 64)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 48, 48, 64)       

In [ ]:

# Train the model (replace train_generator and validation_generator with your actual generators)
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator
)


Epoch 1/20
3/3 [==============================] - 11s 3s/step - loss: 3.2691 - accuracy: 0.1071 - val_loss: 3.5304 - val_accuracy: 0.0588
Epoch 2/20
3/3 [==============================] - 9s 3s/step - loss: 3.0898 - accuracy: 0.1310 - val_loss: 3.4957 - val_accuracy: 0.0882
Epoch 3/20
3/3 [==============================] - 9s 3s/step - loss: 3.0482 - accuracy: 0.1310 - val_loss: 3.4744 - val_accuracy: 0.1176
Epoch 4/20
3/3 [==============================] - 11s 4s/step - loss: 2.8286 - accuracy: 0.1905 - val_loss: 3.4524 - val_accuracy: 0.1176
Epoch 5/20
3/3 [==============================] - 9s 3s/step - loss: 2.7506 - accuracy: 0.2024 - val_loss: 3.4105 - val_accuracy: 0.1176
Epoch 6/20
3/3 [==============================] - 9s 3s/step - loss: 2.6816 - accuracy: 0.2143 - val_loss: 3.3538 - val_accuracy: 0.1471
Epoch 7/20
3/3 [==============================] - 11s 4s/step - loss: 2.3068 - accuracy: 0.3452 - val_loss: 3.3734 - val_accuracy: 0.1765
Epoch 8/20
3/3 [======================

In [ ]:

# Evaluate the model on the validation data
scores = model.evaluate(validation_generator)
print(f"Validation accuracy: {scores[1]*100:.2f}%")


2/2 [==============================] - 2s 63ms/step - loss: 3.5495 - accuracy: 0.0588
Validation accuracy: 5.88%


## Prediction on testset

In [ ]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def predict_single_image(model, image_path, target_size, class_labels):
    image = load_img(image_path, target_size=target_size)
    image_array = img_to_array(image)
    image_array = np.expand_dims(image_array, axis=0)
    prediction = model.predict(image_array)
    predicted_class_index = np.argmax(prediction, axis=-1)[0]
    predicted_class_label = class_labels[predicted_class_index]
    return predicted_class_label

def predict_multiple_images(model, folder_path, class_labels, target_size=(200, 200)):
    predicted_labels = {}
    for root, dirs, files in os.walk(folder_path):
        for image_filename in files:
            # Construct the full path to the image file
            image_path = os.path.join(root, image_filename)
            if os.path.isfile(image_path):  # Ensure that the item is a file
                predicted_class_label = predict_single_image(model, image_path, target_size, class_labels)
                predicted_labels[image_filename] = predicted_class_label
    return predicted_labels

# Define the path to the folder containing the test images
test_dir = '/content/drive/Shareddrives/BA865 - Team Project/Image_dataset/test'

# Get the class labels from the training generator (replace train_generator with your actual generator)
class_labels = list(train_generator.class_indices.keys())



In [ ]:
# Use the trained model to predict the classes of the test images
predicted_labels = predict_multiple_images(model, test_dir, class_labels)
print('Predicted class labels for test images:')
for image_filename, predicted_class_label in predicted_labels.items():
    print(f'{image_filename}: {predicted_class_label}')

1/1 [==============================] - 0s 58ms/step
Predicted class labels for test images:
yes.0.png: rain
eat.0.png: eat
